In [1]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
import requests # Importing requests
import re # Importing regex
from selenium import webdriver
# Importing required exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings("ignore")

### 1. Write a python program which searches all the product under a particular product from www.amazon.in.  The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars. 

In [2]:
driver = webdriver.Chrome("chromedriver.exe")

In [3]:
url = "https://www.amazon.in/ref=nav_logo"
driver.get(url)

In [4]:
search = input ("Enter the product name you want to search")

Enter the product name you want to searchguitar


In [5]:
search_product = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
search_product.clear()
search_product.send_keys(search)
search_btn = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search_btn.click()

### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.
 

In [6]:
urls = []

for list  in range(0,3):
    
    try:
        list_page = driver.find_elements_by_xpath('//a[@class="a-link-normal s-link-style a-text-normal"]')
        
        for url in list_page:
            url = url.get_attribute("href")
            
            if url[0:4]=='http':
                urls.append(url)
        print("Products of page {} has been scraped.".format(list+1))
        
        nxt_btn = driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
        if nxt_btn.text == 'Next':
            nxt_btn.click()
            time.sleep(4)
        elif driver.find_element_by_xpath('a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').text=='Next':
            print("No New Page Exist")
            break
            
    except NoSuchElementException as e:
        print("No Such Element Exception")
        nxt_page = nxt_btn.get_attribute('href')
        driver.get(nxt_page)

Products of page 1 has been scraped.
Products of page 2 has been scraped.
Products of page 3 has been scraped.


In [7]:
len(urls)

159

In [8]:
brand_names = []
product_names = []
prices = []
returns = []
expected_deliveries = []
availabilities = []
product_urls = []

for url in urls:
    driver.get(url)
    print("Scraped URL is =",url)
    
    try:
        brand_name = driver.find_element_by_xpath('//a[@id="bylineInfo"]')
        brand_names.append(brand_name.text)
    except NoSuchElementException:
        brand_names.append('-')
        
    try:
        product_name = driver.find_element_by_xpath('//h1[@id="title"]/span')
        product_names.append(product_name.text)
    except NoSuchElementException:
        product_names.append('-')
        
    try:
        price = driver.find_element_by_xpath('//span[@class="a-price aok-align-center priceToPay"]')
        prices.append(price.text.replace('\n','.'))
    except NoSuchElementException:
        prices.append('-')
        
    try:
        returns_exchange = driver.find_element_by_xpath('//div[@class="a-section a-spacing-none icon-content"]')
        returns.append(returns_exchange.text)
    except NoSuchElementException:
        returns.append('-')
        
    try:
        delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')
        expected_deliveries.append(delivery.text)
    except NoSuchElementException:
        expected_deliveries.append('-')
        
    try:
        availability = driver.find_element_by_xpath('//div[@id="availability"]/span')
        availabilities.append(availability.text)
    except NoSuchElementException:
        availabilities.append('-')
        
    product_urls.append(url)
    time.sleep(2)

Scraped URL is = https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A02055132SF64AM9MYQQ8&url=%2FKadence-6-strings-Acoustic-Guitar-Natural%2Fdp%2FB07JKYWGJN%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dguitar%26qid%3D1642874081%26sr%3D8-1-spons%26psc%3D1%26smid%3DAM04Z7MH4HSDD&qualifier=1642874081&id=6239770463190160&widgetName=sp_atf
Scraped URL is = https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A023421427JVP0SLXXND&url=%2FJOY-TOY-Enterprise-4-String-Acoustic%2Fdp%2FB098JWG1HD%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dguitar%26qid%3D1642874081%26sr%3D8-2-spons%26psc%3D1&qualifier=1642874081&id=6239770463190160&widgetName=sp_atf
Scraped URL is = https://www.amazon.in/Juarez-Acoustic-Cutaway-038C-Strings/dp/B017NPCSLI/ref=sr_1_3?keywords=guitar&qid=1642874081&sr=8-3
Scraped URL is = https://www.amazon.in/Juarez-Acoustic-Guitar-Cutaway-Strings/dp/B076QHZ4HZ/ref=sr_1_4?keywords=guitar&qid=1642874081&sr=8-4
Scraped

In [10]:
df = pd.DataFrame({'Brand Name':brand_names,'Product Name':product_names,'Price':prices,'Return/Exchange':returns,'Expected Delivery':expected_deliveries,'Availability':availabilities,'Product URL':product_urls})
df.head(10)

,Brand Name,Product Name,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Visit the Kadence Store,"Kadence Guitar Acoustica Series, 41” Jumbo Siz...",-,Pay on Delivery,"Thursday, Jan 27",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Brand: JOY&TOY,JOY&TOY Enterprise 4-String Acoustic Guitar Le...,₹989.00,7 Days Replacement,Jan 29 - Feb 4,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹2,649.00",Pay on Delivery,"Thursday, Jan 27",In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
3,Visit the JUAREZ Store,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...","₹2,190.00",Pay on Delivery,"Thursday, Jan 27",In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
4,Visit the Intern Store,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...","₹2,350.00",Pay on Delivery,"Thursday, Jan 27",In stock.,https://www.amazon.in/Intern-INT-38C-Acoustic-...
5,Visit the Intern Store,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...","₹2,350.00",Pay on Delivery,"Thursday, Jan 27",In stock.,https://www.amazon.in/Intern-INT-38C-Acoustic-...
6,Visit the Kadence Store,Kadence Frontier guitar with Online Guitar lea...,-,Pay on Delivery,"Friday, Jan 28",In stock.,https://www.amazon.in/Kadence-Frontier-Acousti...
7,Visit the JUAREZ Store,"Juarez Acoustic Guitar Kit, 38 Inch Cutaway, 3...","₹2,649.00",Pay on Delivery,"Monday, Jan 31",In stock.,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
8,Brand: Epiphone,Epiphone Les Paul SL 6 Strings Right Handed El...,"₹12,999.00",Pay on Delivery,"Monday, Jan 31",Only 2 left in stock.,https://www.amazon.in/Epiphone-ENOLVSCH1-Elect...
9,Visit the JUAREZ Store,JUAREZ JRZ38C Right Handed Acoustic Guitar wit...,"₹2,649.00",Pay on Delivery,"Friday, Jan 28",In stock.,https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-N...


### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [11]:
driver = webdriver.Chrome("chromedriver.exe")

In [17]:
url = "https://images.google.com/"

urls = []    
data = []    
search_item = ["fruits", "cars", "Machine Learning", "Guitar", "Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    
    search_bar = driver.find_element_by_tag_name("input") 
    
    
    search_bar.send_keys(str(item))
    

    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click()
    
    
    for _ in range(100):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:50]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 50:
        break
    print("Downloading {0} of {1} images" .format(i, 50))
    response = requests.get(urls[i])

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.
 

In [50]:
driver = webdriver.Chrome("chromedriver.exe")

In [51]:
url = "https://www.flipkart.com/"
driver.get(url)

In [52]:
search_bar = driver.find_element_by_xpath('//input[@class="_3704LK"]')
search_bar.send_keys("Pixel 4A")

In [53]:
search_btn = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')
search_btn.click()

In [54]:
urls = []

url_1 = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

for url in url_1:
    urls.append(url.get_attribute('href'))

In [55]:
len(urls)

24

In [56]:
brand_names = []
smartphone_name = []
colour = []
ram = []
storage = []
primary = []
secondary = []
display = []
battery = []
price = []
product_url = []

for url in urls:
    driver.get(url)
    print("Scrapping Url=",url)
    product_url.append(url)
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    try:
        brand_name = driver.find_element_by_xpath('//span[@class="B_NuCI"]')
        brand_names.append(brand_name.text.split()[0])
    except NoSuchElementException:
        brand_names.append('-')
        
    try:
        smartphone = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')
        smartphone_name.append(smartphone.text)
    except NoSuchElementException:
        smartphone_name.append('-')
        
    try:
        colour_name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')
        colour.append(colour_name.text)
    except NoSuchElementException:
        colour.append('-')
        
    try:
        ram_type = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table/tbody/tr[2]/td[2]/ul/li')
        ram.append(ram_type.text)
    except NoSuchElementException:
        ram.append('-')
        
    try:
        storage_type = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table/tbody/tr[1]/td[2]/ul/li')
        storage.append(storage_type.text)
    except NoSuchElementException:
        storage.append('-')
        
    try:
        primary_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table/tbody/tr[2]/td[2]/ul/li')
        primary.append(primary_cam.text)
    except NoSuchElementException:
        primary.append('-')
        
    try:
        secondary_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table/tbody/tr[6]/td[2]/ul/li')
        secondary.append(secondary_cam.text)
    except NoSuchElementException:
        secondary.append('-')
        
    try:
        display_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table/tbody/tr[1]/td[2]/ul/li')
        display.append(display_size.text)
    except NoSuchElementException:
        display.append('-')
        
    try:
        battery_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[1]/td[2]/ul/li')
        battery.append(battery_cap.text)
    except NoSuchElementException:
        battery.append('-')
        
    try:
        prices = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')
        price.append(prices.text)
    except NoSuchElementException:
        price.append('-')

Scrapping Url= https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUNVSP4A&marketplace=FLIPKART&q=Pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=be1eb5e9-5837-43e6-bd2a-fd5efaec6a67.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=40kjt6x29c0000001643228654877&qH=aeec4064cb2b1348
Scrapping Url= https://www.flipkart.com/redmi-9a-seablue-32-gb/p/itmeabd39a0cd669?pid=MOBFVD9ZGTYR2WGR&lid=LSTMOBFVD9ZGTYR2WGRIGIQ6G&marketplace=FLIPKART&q=Pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=be1eb5e9-5837-43e6-bd2a-fd5efaec6a67.MOBFVD9ZGTYR2WGR.SEARCH&ppt=hp&ppn=homepage&ssid=40kjt6x29c0000001643228654877&qH=aeec4064cb2b1348
Scrapping Url= https://www.flipkart.com/redmi-9a-midnight-black-32-gb/p/itmeabd39a0cd669?pid=MOBFVDA2RWTCFBHZ&lid=LSTMOBFVDA2RWTCFBHZJV9QUN&marketplace=FLIPKART&q=Pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search&fm=SEAR

In [57]:
df = pd.DataFrame({'Brand Name':brand_names,'Smartphone Name':smartphone_name,'Colour':colour,'RAM':ram,'Storage (ROM)':storage,'Primary Camera':primary,'Secondary Camera':secondary,'Display Size':display,'Battery Capacity':battery,'Price':price,'Product URL':product_url})
df

,Brand Name,Smartphone Name,Colour,RAM,Storage (ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,Product URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,"8MP Front Camera, 1.12 μm Pixel Size, F/2.0 Ap...",14.76 cm (5.81 inch),69.4 mm,"₹27,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Redmi,Redmi 9A,SeaBlue,2 GB,32 GB,-,-,16.59 cm (6.53 inch),1 Year Manufacturer Warranty,"₹7,717",https://www.flipkart.com/redmi-9a-seablue-32-g...
2,Redmi,Redmi 9A,Midnight Black,2 GB,32 GB,13MP Rear Camera,Yes,16.59 cm (6.53 inch),1 Year Manufacturer Warranty,"₹7,724",https://www.flipkart.com/redmi-9a-midnight-bla...
3,Redmi,Redmi 9A,Nature Green,2 GB,32 GB,13MP Rear Camera,Yes,16.59 cm (6.53 inch),1 Year Manufacturer Warranty,"₹7,689",https://www.flipkart.com/redmi-9a-nature-green...
4,Redmi,Redmi 9A Sport,Carbon Black,2 GB,32 GB,13MP Rear Camera,Back,16.59 cm (6.53 inch),5000 mAh,"₹7,769",https://www.flipkart.com/redmi-9a-sport-carbon...
5,REDMI,REDMI NOTE 10 LITE,Glacier White,4 GB,128 GB,Primary Camera,-,16.94 cm (6.67 inch),-,"₹14,995",https://www.flipkart.com/redmi-note-10-lite-gl...
6,Redmi,Redmi 9,Sporty Orange,4 GB,64 GB,13MP + 2MP,"AI Beautify 5.0 , AI portrait mode , HDR , Scr...",16.59 cm (6.53 inch),5000 mAh,"₹10,790",https://www.flipkart.com/redmi-9-sporty-orange...
7,SAMSUNG,Galaxy M52 5G,Blazing Black,6 GB,128 GB,64MP + 12MP + 5MP,Smart Selfie Angle,17.02 cm (6.7 inch),1 Year Warranty Provided by the Manufacturer f...,"₹26,997",https://www.flipkart.com/samsung-galaxy-m52-5g...
8,REDMI,10 Prime,Astral White,4 GB,64 GB,50MP + 8MP + 2MP + 2MP,Yes,16.51 cm (6.5 inch),6000 mAh,"₹14,490",https://www.flipkart.com/redmi-10-prime-astral...
9,Tecno,Spark 7,Magnet Black,2 GB,32 GB,16MP Rear Camera,Yes,16.56 cm (6.52 inch),6000 mAh,"₹7,975",https://www.flipkart.com/tecno-spark-7-magnet-...


In [58]:
df.to_csv("flipkart_phone.csv")

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.
 

In [76]:
driver = webdriver.Chrome("chromedriver.exe")

In [77]:
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)

City = input("Enter the city name:")
search_title = driver.find_element_by_id('searchboxinput')
search_title.clear()
time.sleep(2)

search_title.send_keys(City)

search_btn = driver.find_element_by_xpath('//div[@class="xoLGzf-BIqFsb-haAclf"]')
search_btn.click()

try:
    urls = driver.current_url
    print("URL is:",urls)
    latitude_longitude = re.findall(r'@(.*)',urls)
    if len(latitude_longitude):
        latitude_list = latitude_longitude[0].split(',')
        if len(latitude_list)>=2:
            latitude = latitude_list[0]
            longitude = latitude_list[1]
        print("latitude={}, longitude={}".format(latitude,longitude))
        
except Exception as e:
    print("Error:",str(e))

Enter the city name:delhi
URL is: https://www.google.co.in/maps/@25.4178681,86.1250335,14z
latitude=25.4178681, longitude=86.1250335


In [78]:
df.to_csv("City_Geospatial.csv")

### 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)  from trak.in. 

In [2]:
driver = webdriver.Chrome("chromedriver.exe")

In [3]:
url = "https://trak.in/"
driver.get(url)

In [4]:
fund_deals = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_deals)

In [40]:
Date = []
Startup = []
Industry = []
SubVertical = []
Location = []
Investor = []
Investment = []
Amount = []

for i in range(54,57):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup.append(n.text)
    
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)
    
    subvertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in subvertical:
        SubVertical.append(s.text)

    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
    
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
    amtount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amtount:
        Amount.append(a.text)

In [42]:
df = pd.DataFrame({'Date':Date,'Startup Name':Startup,'Industry/Vertical':Industry,'Sub Vertical':SubVertical,'Location':Location,'Investor Name':Investor,'Investor Type':Investment,'Amount':Amount})
df

,Date,Startup Name,Industry/Vertical,Sub Vertical,Location,Investor Name,Investor Type,Amount
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [43]:
df.to_csv("Trak_in.csv")

### 7. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [68]:
driver = webdriver.Chrome("chromedriver.exe")

In [72]:
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

laptop_btn = driver.find_element_by_xpath('/html/body/div[2]/div/div[4]/ul/li[3]/a')
laptop_btn.click()
time.sleep(2)

best_gaming =driver.find_element_by_xpath('/html/body/div[7]/div/div[2]/div[2]/ul/li[10]/a')
best_gaming.click()
time.sleep(2)

Laptop_Name = []
OS = []
display = []
processor = []
memory = []
weight = []
dimension = []
graphics = []
Price = []

laptop = driver.find_elements_by_xpath('//div[@class="right-container"]/div/a/h3')
for name in laptop:
    Laptop_Name.append(name.text)
    
try:
    operating = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[3]/td[3]')
    for system in operating:
        OS.append(system.text)
except NoSuchElementException:
    pass

try:
    dis = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[4]/td[3]')
    for size in dis:
        display.append(size.text)
except NoSuchElementException:
    pass

try:
    pro = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[5]/td[3]')
    for p in pro:
        processor.append(p.text)
except NoSuchElementException:
    pass

try:
    mem = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[6]/td[3]')
    for m in mem:
        memory.append(m.text)
except NoSuchElementException:
    pass

try:
    wgt = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[7]/td[3]')
    for w in wgt:
        weight.append(w.text)
except NoSuchElementException:
    pass

try:
    dim = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[8]/td[3]')
    for d in dim:
        dimension.append(d.text)
except NoSuchElementException:
    pass

try:
    Grp = driver.find_elements_by_xpath('//div[@class="Spcs-details"][1]/table/tbody/tr[9]/td[3]')
    for g in Grp:
        graphics.append(g.text)
except NoSuchElementException:
    pass

try:
    pri = driver.find_elements_by_xpath('//td[@class="smprice"]')
    for pr in pri:
        Price.append(pr.text)
except NoSuchElementException:
    pass

In [73]:
print(len(Laptop_Name),len(OS),len(display),len(processor),len(memory),len(weight),len(dimension),len(graphics),len(Price))

10 10 10 10 10 10 10 10 10


In [74]:
df = pd.DataFrame({'Laptop Name':Laptop_Name,'Operating System':OS,'Display Size':display,'Processor':processor,'Memory':memory,'Weight':weight,'Dimension':dimension,'Graphics':graphics,'Price':Price})
df

,Laptop Name,Operating System,Display Size,Processor,Memory,Weight,Dimension,Graphics,Price
0,ACER NITRO 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"₹ 129,990"
1,MSI STEALTH 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"₹ 134,990"
2,ASUS ROG STRIX SCAR 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"₹ 268,990"
3,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Gen Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"₹ 342,989"
4,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Gen Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"₹ 319,990"
5,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"₹ 215,990"
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"₹ 164,990"
7,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"₹ 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"₹ 185,000"
9,ACER ASPIRE 7 GAMING LAPTOP,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"₹ 57,900"


In [75]:
df.to_csv("Digit_Gaming_Laptops.csv")

# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”

In [14]:
driver = webdriver.Chrome("chromedriver.exe")

In [18]:
url = "https://www.forbes.com/?sh=7bf36e032254"
driver.get(url)
time.sleep(2)

explore_btn = driver.find_element_by_xpath('//button[@class="icon--hamburger"]')
explore_btn.click()
time.sleep(1)

billion_btn = driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]')
billion_btn.click()
time.sleep(1)

world_billion = driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
world_billion.click()
time.sleep(2)

In [19]:
Rank = []
Name = []
Net_worth = []
Age = []
Citizenship = []
Source = []
Industry = []

while(True):
    ranks = driver.find_elements_by_xpath('//div[@class="rank"]')
    for r in ranks:
        Rank.append(r.text)
    time.sleep(1)

    names = driver.find_elements_by_xpath('//div[@class="personName"]/div')
    for n in names:
        Name.append(n.text)
    time.sleep(1)

    net = driver.find_elements_by_xpath('//div[@class="netWorth"]//div[1]')
    for ne in net:
        Net_worth.append(ne.text)
    time.sleep(1)

    ages = driver.find_elements_by_xpath('//div[@class="age"]/div')
    for a in ages:
        Age.append(a.text)
    time.sleep(1)

    citizen = driver.find_elements_by_xpath('//div[@class="countryOfCitizenship"]')
    for c in citizen:
        Citizenship.append(c.text)
    time.sleep(1)

    sources = driver.find_elements_by_xpath('//div[@class="source"]')
    for s in sources:
        Source.append(s.text)
    time.sleep(1)

    indus = driver.find_elements_by_xpath('//div[@class="category"]//div')
    for i in indus:
        Industry.append(i.text)
    time.sleep(1)

    try:
        nxt_btn = driver.find_element_by_xpath('//button[@class="pagination-btn pagination-btn--next "]')
        nxt_btn.click()
    except:
        break
    
total_net_worth = []
for t in range(0,len(Net_worth),2):
    total_net_worth.append(Net_worth[t])

In [26]:
df = pd.DataFrame({'Rank':Rank,'Name':Name,'Net Worth':total_net_worth,'Age':Age,'Citizenship':Citizenship,'Source':Source,'Industry':Industry})
df

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


In [27]:
df.to_csv("Forbes_Billionaires_list.csv")

### 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [44]:
driver = webdriver.Chrome("chromedriver.exe")

In [45]:
url = "https://www.youtube.com/"
driver.maximize_window()
driver.get(url)

In [46]:
search_bar = driver.find_element_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')
search_bar.send_keys("Excuses")
time.sleep(2)

search_btn = driver.find_element_by_xpath('/html/body/ytd-app/div/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button')
search_btn.click()
time.sleep(2)

video = driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div/div[1]/div/h3/a/yt-formatted-string')
video.click()
time.sleep(2)

In [50]:
i = 0
while(i<100):
    driver.execute_script("window.scrollBy(0,5000)")
    i+=1
while(i<402):
    driver.execute_script("window.scrollBy(0,10000)")
    i+=1

In [53]:
comments = []
upvote = []
time = []

try:
    comment = driver.find_elements_by_xpath('//yt-formatted-string[@id="content-text"]')
    for c in comment:
        comments.append(c.text)
except NoSuchElementException:
    comments.append('-')
    
try:
    vote = driver.find_elements_by_xpath('//span[@id="vote-count-middle"]')
    for v in vote:
        upvote.append(v.text)
except NoSuchElementException:
    upvote.append('-')
    
try:
    ago = driver.find_elements_by_xpath('//yt-formatted-string[@class="published-time-text above-comment style-scope ytd-comment-renderer"]//a')
    for a in ago:
        time.append(a.text)
except NoSuchElementException:
    time.append('-')

In [54]:
print(len(comments),len(upvote),len(time))

534 534 534


In [55]:
df = pd.DataFrame({'Comments':comments[:500],'Comment Upvote':upvote[:500],'Time When Commented':time[:500]})
df

,Comments,Comment Upvote,Time When Commented
0,0:50 Thanks for appreciating my verse \n\nI am...,9.6K,5 days ago (edited)
1,"Listing in 2022 , and it's amazing to listen i...",1K,1 day ago
2,1 year passed but still this song hits differe...,463,1 day ago
3,"10 years down the line, in a reunion with coll...",83,13 hours ago
4,1.5 year passed but still this song hits diffe...,188,1 day ago (edited)
...,...,...,...
495,Even San Francisco wifey likes this one ...the...,93,1 year ago
496,Love this song,,1 day ago
497,Love this song,,5 hours ago
498,Literally Masterpiece ❤️,,37 seconds ago


In [56]:
df.to_csv("Youtube_comments.csv")

### 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description. 

In [18]:
driver = webdriver.Chrome("chromedriver.exe")

In [26]:
url = "https://www.hostelworld.com/"
driver.get(url)

In [30]:
search_bar = driver.find_element_by_xpath('//input[@id="search-input-field"]')
search_bar.send_keys("London")
time.sleep(2)

london_key = driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]')
london_key.click()
time.sleep(2)

search_btn = driver.find_element_by_xpath('//div[@class="search-button"]')
search_btn.click()
time.sleep(3)

In [35]:
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url=[]

In [44]:
for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(4)
    
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    
    
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):

        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')

    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
        
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
          
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
    
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
    
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
     
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [45]:
len(hostel_name),len(distance),len(pvt_prices),len(dorms_price),len(rating),len(reviews),len(over_all),len(facilities),len(description)

(30, 30, 30, 30, 90, 90, 90, 81, 90)

In [46]:
hostel_name = hostel_name[0:30]
distance = distance[0:30]
pvt_prices = pvt_prices[0:30]
dorms_price = dorms_price[0:30]
rating = rating[0:30]
reviews = reviews[0:30]
over_all = over_all[0:30]
facilities = facilities[0:30]
description =description[0:30]
product_url = product_url[0:30]

In [47]:
Hostel = pd.DataFrame({})
Hostel['Hostel_Name'] = hostel_name
Hostel['Distance fron city centre'] = distance
Hostel['Ratings'] = rating
Hostel['Total_reviews'] = reviews
Hostel['Overall Reviews'] = over_all
Hostel['Privates from price'] = pvt_prices
Hostel['Dorms from price'] = dorms_price
Hostel['Description'] = description
Hostel

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Description
0,Mornington Camden,4.1km from city centre,8.3,44,Fabulous,Rs5511,Rs3266,"2 Hurdwick Place, London, England"
1,Smart Russell Square Hostel,2.6km from city centre,6.7,9567,Good,Rs5511,Rs3266,"71 Guilford Street, London, England"
2,Smart Camden Inn Hostel,4.4km from city centre,8.5,2739,Fabulous,Rs5511,Rs3266,"55/57 Bayham Street, Camden, London, England"
3,Selina Camden,5.5km from city centre,9.1,14,Superb,Rs5511,Rs3266,"89 Chalk Farm Rd, Chalk Farm, London, England"
4,Queen Elizabeth Chelsea,5.7km from city centre,7.5,3214,Very Good,Rs5511,Rs3266,"58 Bagleys Lane, London, England"
5,Pickwick Hall,2.3km from city centre,9.0,2276,Superb,Rs5511,Rs3266,"7 Bedford Place, Bloomsbury, London, England"
6,New Cross Inn Hostel,6.1km from city centre,7.4,1055,Very Good,Rs5511,Rs3266,"323 New Cross Road, London, England"
7,No.8 Willesden Hostel London,10km from city centre,7.6,4645,Very Good,Rs5511,Rs3266,"305 High Rd, Willesden, London, England"
8,Safestay London Elephant & Castle,1.7km from city centre,8.0,4026,Fabulous,Rs5511,Rs3266,"144-152 Walworth Road, London, England"
9,Barmy Badger Backpackers,5.5km from city centre,9.3,1688,Superb,Rs5511,Rs3266,"17 Longridge Road, Earls Court, London, England"


In [49]:
Hostel.to_csv("Hoste_World.csv")